In this demo, we are going to build a convolutional neural network using Keras
Download cat or dog data set from kaggle

In [1]:
train_dir = 'data/train'
test_dir  = 'data/test'
nb_train_samples = 400
nb_test_samples = 400
img_width = 227
img_height = 227

Here, we build the convolutional neural network that will be used to classify the images. This is how a basic convnet can look like:

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D

model = Sequential()

# Most of these values like nb_filters, nb_rows, nb_columns are mostly trial and error.
# There is no definite answer to what is the correct one, just keep trying.
# input_shape[0] = 3 because of RGB channels
model.add(Convolution2D(32, 5, 5, input_shape=(3, img_width, img_height)))

# ReLU ensures none of the output value from CONV layer falls below 0
# To prevent vanishing gradient problem (failure to learn) from happening
model.add(Activation('relu'))

# For every 2x2 "pixels" in the output feature map, pick the highest value,
# more like pick the most activated one.
# Most importantly, pooling effectively reduces the feature map size by 75%
model.add(MaxPooling2D(pool_size=(2,2)))

# Repeat a few times before going into fully connected (dense) layers
# You are free to comment them out or add more conv relu pool layers
# See what it does to the accuracy
model.add(Convolution2D(64, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(128, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(256, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# We can do as many convolutions and poolings as we like, but it's time to really classify
# Again, you can do as many dense layers as you like, use different activations and dropout %
# It's all trial and error, this demo code may not be the most optimal after all!
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Final layer, 4 nodes representing each class, softmax activation makes sure every node sums up to 1
# That means the output nodes are in % score for each food.
model.add(Dense(1))
model.add(Activation('sigmoid'))

# The architecture is there. But how to measure how close we are to perfection?
# Measure the difference from prediction to ground truth - the loss
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


## Prepare image data generator

The inputs for neural networks are tensors - think matrices of numbers. In case of images, the tensors can be rows and columns of pixels, with each pixel quantised into red, green, blue channels. Hence, the 3 layers of convolutional neural networks, along with its fixed widths and heights.

Keras provides this nifty data generator function to generate tensors of images during training. There are plenty of options that can be passed into the generator to augment images in real time. However, we won't be touching those yet.

By simply passing in the directory names into the generator, you saved yourself from:
* Manually generating RGB matrices from images
* Resizing them to reduce processing on neural network's end (full-size photos may not fit into your graphics card VRAM)
* Reshuffle them in every epoch

In [3]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

training_generator = datagen.flow_from_directory(train_dir,
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode="categorical")

testing_generator = datagen.flow_from_directory(test_dir,
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode="categorical")

Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


## LET'S TRAIN THE CONVNET!

Since there are 4 food types (classes) to classify, a neural network that doesn't work will return random results - and yield 0.25 accuracy. So if you network keeps yielding 0.25 `val_acc` after a few epochs, check your setup.


In [4]:
model.fit_generator(
        training_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=20,
        validation_data=testing_generator,
        nb_val_samples=nb_test_samples)

Epoch 1/20
400/400 [==============================] - 15s - loss: 1.4080 - acc: 0.2925 - val_loss: 1.2423 - val_acc: 0.4250
Epoch 2/20
400/400 [==============================] - 12s - loss: 1.2273 - acc: 0.4000 - val_loss: 1.1230 - val_acc: 0.5075
Epoch 3/20
400/400 [==============================] - 12s - loss: 1.1432 - acc: 0.4950 - val_loss: 1.0112 - val_acc: 0.6575
Epoch 4/20
400/400 [==============================] - 12s - loss: 1.0288 - acc: 0.5725 - val_loss: 0.9660 - val_acc: 0.6850
Epoch 5/20
400/400 [==============================] - 12s - loss: 0.9778 - acc: 0.6075 - val_loss: 0.9728 - val_acc: 0.6375
Epoch 6/20
400/400 [==============================] - 12s - loss: 1.0565 - acc: 0.5600 - val_loss: 1.1964 - val_acc: 0.5425
Epoch 7/20
400/400 [==============================] - 12s - loss: 1.0494 - acc: 0.5700 - val_loss: 1.2128 - val_acc: 0.5125
Epoch 8/20
400/400 [==============================] - 12s - loss: 1.0617 - acc: 0.5850 - val_loss: 0.9337 - val_acc: 0.6925
Epoch 9/

## Let's try augment the image data for higher accuracy

In previous image data generator, we simply ask it to stream from the photo folders, and pass the photo at a smaller size into the convnet. We can reach about 70% validation accuracy - that means every 10 times the network is shown a photo they haven't seen before, the network got it right 7 times.

A few things we can do to _maybe_ make the training better:

* Randomly rotate the images left and right because people don't always take food photos upright?
* Randomly flip the photo horizontally?

Keras built-in ImageDataGenerator is able to augment these images at training time, this is extremely handy when you have little training data, but you want your model to be able to generalise for wider range of applications - to look at different food photos better.

#### How to know if rotating and flipping images is working?
Look at the increase of `val_acc` and drop of `val_loss` compared to the log above. That means the neural network is able to generalise for the photos it has never looked at before.


In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    horizontal_flip=True)

training_generator = datagen.flow_from_directory(train_dir,
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode="categorical")

testing_generator = datagen.flow_from_directory(test_dir,
    target_size=(img_width, img_height),
    batch_size=20,
    class_mode="categorical")

model.fit_generator(
        training_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=20,
        validation_data=testing_generator,
        nb_val_samples=nb_test_samples)

Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Epoch 1/20
400/400 [==============================] - 12s - loss: 0.7468 - acc: 0.7325 - val_loss: 0.8976 - val_acc: 0.6200
Epoch 2/20
400/400 [==============================] - 12s - loss: 0.7991 - acc: 0.6950 - val_loss: 0.8565 - val_acc: 0.6500
Epoch 3/20
400/400 [==============================] - 12s - loss: 0.7128 - acc: 0.7350 - val_loss: 0.9700 - val_acc: 0.6375
Epoch 4/20
400/400 [==============================] - 12s - loss: 0.6637 - acc: 0.7425 - val_loss: 0.8659 - val_acc: 0.6850
Epoch 5/20
400/400 [==============================] - 12s - loss: 0.5849 - acc: 0.7975 - val_loss: 0.7808 - val_acc: 0.7275
Epoch 6/20
400/400 [==============================] - 12s - loss: 0.6010 - acc: 0.7800 - val_loss: 0.8635 - val_acc: 0.7050
Epoch 7/20
400/400 [==============================] - 13s - loss: 0.6888 - acc: 0.7150 - val_loss: 0.6902 - val_acc: 0.7200
Epoch 8/20
400/400 [==============================

## It worked!

Our dataset is really small, with only 4 types of food and only 100 photos per category to train, `ImageDataGenerator` really is our friend for rotating and flipping our images around so our convolutional neural network can learn to classify foods better - with a peak validating accuracy of 80%!

## Explore more

1. [CS231n - Convolutional Neural Net](http://cs231n.github.io/convolutional-networks/#fc)
2. [Keras homepage](https://keras.io/layers)